# Prática: Redes Neurais Convolucionais

Vamos agora implementar a rede [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), uma das redes que trouxeram todo esse interesse para a área de *deep learning*.





In [1]:
!pip install mxnet-cu100

# imports basicos
import time, os, sys, numpy as np
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, utils as gutils, data as gdata

# Tenta encontrar GPU
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,), ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

ctx = try_gpu()
ctx

## carregando dados

# código para carregar o dataset do MNIST
# http://yann.lecun.com/exdb/mnist/
def load_data_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'mnist')):
    """Download the MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.MNIST(root=root, train=True)
    mnist_test = gdata.vision.MNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# código para carregar o dataset do CIFAR 10
# https://www.cs.toronto.edu/~kriz/cifar.html
def load_data_cifar10(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'cifar10')):
    """Download the MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.CIFAR10(root=root, train=True)
    mnist_test = gdata.vision.CIFAR10(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter
  
# funções básicas
def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n, l = nd.array([0]), 0, 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            # X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            y = y.astype('float32')
            y_hat = net(X)
            l += loss(y_hat, y).sum()
            acc_sum += (y_hat.argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n, l.asscalar() / n
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, ctx,
                   num_epochs):
    print('training on', ctx)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss, ctx)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

## AlexNet

Agora já temos todo o conhecimento necessário para implementar nossa primeira arquitetura moderna.
Vamos implementar a [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), uma das arquiteturas mais famosas dessa nova onda de rede neurais.

<p align="center">
  <img width=700 src="https://www.researchgate.net/profile/Jaime_Gallego2/publication/318168077/figure/fig1/AS:578190894927872@1514862859810/AlexNet-CNN-architecture-layers.png">
</p>

<p align="center">
  <img width=700 src="https://engmrk.com/wp-content/uploads/2018/10/AlexNet_Summary_Table.jpg">
</p>


In [0]:
class AlexNet(nn.HybridBlock):
    r"""AlexNet model from the `"One weird trick..." `_ paper.

    Parameters
    ----------
    classes : int, default 10
        Number of classes for the output layer.
    """
    def __init__(self, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        with self.name_scope():
            self.features = nn.HybridSequential(prefix='')
            with self.features.name_scope():
                self.features.add(nn.Conv2D(96, kernel_size=11, strides=4, padding=0, activation='relu'))  # entrada: (b, 3, 227, 227) e saida: (b, 96, 55, 55)
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))                                    # entrada: (b, 96, 55, 55) e saida: (b, 96, 27, 27)
                self.features.add(nn.Conv2D(256, kernel_size=5, padding=2, activation='relu'))             # entrada: (b, 96, 27, 27) e saida: (b, 256, 27, 27)
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))                                    # entrada: (b, 256, 27, 27) e saida: (b, 256, 13, 13)
                self.features.add(nn.Conv2D(384, kernel_size=3, padding=1, activation='relu'))             # entrada: (b, 256, 13, 13) e saida: (b, 384, 13, 13)
                self.features.add(nn.Conv2D(384, kernel_size=3, padding=1, activation='relu'))             # entrada: (b, 384, 13, 13) e saida: (b, 384, 13, 13)
                self.features.add(nn.Conv2D(256, kernel_size=3, padding=1, activation='relu'))             # entrada: (b, 384, 13, 13) e saida: (b, 256, 13, 13)
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))                                    # entrada: (b, 256, 13, 13) e saida: (b, 256, 6, 6)
                self.features.add(nn.Flatten())                                                            # entrada: (b, 256, 13, 13) e saida: (b, 256*6*6) = (b, 9216)
                self.features.add(nn.Dense(4096, activation='relu'))                                       # entrada: (b, 9216) e saida: (b, 4096)
                self.features.add(nn.Dropout(0.5))
                self.features.add(nn.Dense(4096, activation='relu'))                                       # entrada: (b, 4096) e saida: (b, 4096)
                self.features.add(nn.Dropout(0.5))

            self.output = nn.Dense(classes)  # entrada: (b, 4096) e saida: (b, 10)

    def hybrid_forward(self, F, x):
        x = self.features(x)
        x = self.output(x)
        return x

In [5]:
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001
    
net = AlexNet()
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr, 'wd': wd_lambda})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 2.0284, train acc 0.237, test loss 1.7533, test acc 0.358, time 77.1 sec
epoch 2, train loss 1.6700, train acc 0.383, test loss 1.5029, test acc 0.442, time 73.7 sec
epoch 3, train loss 1.4966, train acc 0.451, test loss 1.3802, test acc 0.496, time 74.0 sec
epoch 4, train loss 1.3935, train acc 0.490, test loss 1.3239, test acc 0.518, time 74.0 sec
epoch 5, train loss 1.3150, train acc 0.524, test loss 1.2360, test acc 0.556, time 73.9 sec
epoch 6, train loss 1.2433, train acc 0.553, test loss 1.1952, test acc 0.568, time 73.9 sec
epoch 7, train loss 1.1862, train acc 0.573, test loss 1.1541, test acc 0.591, time 73.6 sec
epoch 8, train loss 1.1419, train acc 0.592, test loss 1.1446, test acc 0.595, time 73.1 sec
epoch 9, train loss 1.1044, train acc 0.609, test loss 1.0795, test acc 0.612, time 73.0 sec
epoch 10, train loss 1.0610, train acc 0.624, test loss 1.0747, test acc 0.619, time 73.0 sec
epoch 11, train loss 1.0252, train acc 0.636, test